In [1]:
import sys
import catboost as cb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import skopt
import scipy
from skopt.space import Real

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    'task_type' : 'GPU',
    'devices' : '0:1',
    'bootstrap_type' : 'MVS',
    'has_time' : True,
    
    #"rsm" : 0.36719138525672734,
    'bagging_temperature': 1.1063407351624084,
    'border_count': 7137,
    'depth': 5,
    'early_stopping_rounds': 4,
     'iterations': 7,
    'l2_leaf_reg': 0.17260178161990627,
    'learning_rate': 0.027307511040591478,
    'random_seed': 5712,
    'random_strength': 2.3464965049108726e-07,
    'scale_pos_weight': 0.819600044656442,
    'subsample': 0.9880746674599722
}

#No escala muy bien a partir del valor 12 
N_CLUSTERS = 7

In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Arboles_entrenamiento.pkl")
test = pd.read_pickle("../Archivos/Arboles_validacion.pkl")

if (PREDICCION_REAL):
    entrenamiento = entrenamiento.append(test)
    test = pd.read_pickle("../Archivos/Arboles_test.pkl")

In [3]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA

#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
fugas = ['Sales_Contract_No','ID','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','ASP','ASP_(converted)']
otros = ['Currency']

entrenamiento = entrenamiento.drop(columns=fugas)
test = test.drop(columns=fugas)

#entrenamiento = entrenamiento.drop(columns=columnas_fecha)
#test = test.drop(columns=columnas_fecha)

In [4]:
#FECHAS A DIAS

columnas_fecha = ['Year-Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
def fecha_a_dias(x):
    fecha_origen = pd.to_datetime('01/01/2000', format='%m/%d/%Y')
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (x - fecha_origen).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

In [5]:
#EXTRACCION DE STAGE

objetivo = entrenamiento['Stage']
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')
    
if not PREDICCION_REAL:
    test_label = test['Stage']
    test = test.drop(columns=['Stage'])

In [6]:
#LIMPIEZA EXTRA PARA CLUSTERING
import category_encoders

def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
            
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

def limpiar_nan(col,entrenamiento,test):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)
    
def limpiar_inf(col,entrenamiento,test):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)
    
def limpiar_nan(col,entrenamiento,test):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)
    
def limpiar_inf(col,entrenamiento,test):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)

def limpiar(entrenamiento,test,limite):
    for col in entrenamiento.columns:
        ent_null = entrenamiento.loc[entrenamiento[col] == np.inf]
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_inf(col,entrenamiento,test)        
    for col in test.columns:
        ent_null = test.loc[test[col] == np.inf]
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_inf(col,entrenamiento,test)        

    for col in entrenamiento.columns:
        ent_null = entrenamiento.loc[entrenamiento[col].isnull()]
        ent_null = ent_null.replace(np.NaN,0)
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_nan(col,entrenamiento,test)        
    for col in test.columns:
        ent_null = test.loc[test[col].isnull()]
        ent_null = ent_null.replace(np.NaN,0)
        if (ent_null[col].count() > limite):
            entrenamiento.drop(columns = [col],inplace=True)
            test.drop(columns = [col],inplace=True)
        if (0<ent_null[col].count() <= limite):
            limpiar_nan(col,entrenamiento,test)

limpiar(entrenamiento,test,1000)

In [7]:
# SEPARACION EN CLUSTERS
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0).fit(entrenamiento)
kmeans.predict(entrenamiento)

entrenamiento['Cluster'] = kmeans.predict(entrenamiento)
test['Cluster'] = kmeans.predict(test)

entrenamiento['Stage'] = objetivo
test['Stage'] = test_label

In [8]:
entrenamiento['Cluster'].value_counts()

0    6411
6    5005
3     454
4     189
2      18
5       3
1       1
Name: Cluster, dtype: int64

In [9]:
test['Cluster'].value_counts()

0    3548
6     762
3     153
4      50
2       2
Name: Cluster, dtype: int64

In [10]:
#DIVISION

model = []
preds = []
slice_entrenamiento = []
slice_test = []
slice_stage_entrenamiento = []
slice_stage_test = []

for i in range (0, N_CLUSTERS):
    print("Cluster " + str(i) + ":")
    
    slice_entrenamiento.append(entrenamiento[entrenamiento['Cluster'] == i].copy())
    slice_test.append(test[test['Cluster'] == i].copy())
    
    slice_entrenamiento[i] = slice_entrenamiento[i].drop(columns='Cluster')
    slice_test[i] = slice_test[i].drop(columns='Cluster')
    
    slice_stage_entrenamiento.append(slice_entrenamiento[i].pop('Stage'))
    slice_stage_test.append(slice_test[i].pop('Stage'))
    
    es_ruido = pd.Series(slice_stage_entrenamiento[i]).mean() > 0.98 or pd.Series(slice_stage_entrenamiento[i]).mean() < 0.02
    
    
    entrenamiento_pool = cb.Pool(slice_entrenamiento[i], slice_stage_entrenamiento[i])
    if(es_ruido):
        test_pool = cb.Pool(slice_test[i])
    else:
        test_pool = cb.Pool(slice_test[i], slice_stage_test[i])
        
    model.append(cb.CatBoostClassifier(**PARAMETROS))
    
    if(es_ruido):
        preds.append(slice_stage_entrenamiento[i])
        print("Cluster con pocos elementos o poca variedad de stage")
        continue
    
    model[i].fit(entrenamiento_pool, eval_set = test_pool)
    preds.append(model[i].predict_proba(test_pool))
    preds[i] =  pd.Series([p[1] for p in  preds[i]])
   

Cluster 0:
0:	learn: 0.6259457	test: 0.6832380	best: 0.6832380 (0)	total: 38.8ms	remaining: 233ms
1:	learn: 0.5678271	test: 0.6747624	best: 0.6747624 (1)	total: 76.8ms	remaining: 192ms
2:	learn: 0.5196699	test: 0.6667335	best: 0.6667335 (2)	total: 119ms	remaining: 159ms
3:	learn: 0.4783762	test: 0.6588669	best: 0.6588669 (3)	total: 162ms	remaining: 122ms
4:	learn: 0.4429015	test: 0.6518958	best: 0.6518958 (4)	total: 207ms	remaining: 82.9ms
5:	learn: 0.4139174	test: 0.6451732	best: 0.6451732 (5)	total: 269ms	remaining: 44.9ms
6:	learn: 0.3895762	test: 0.6391210	best: 0.6391210 (6)	total: 313ms	remaining: 0us
bestTest = 0.6391210301
bestIteration = 6
Cluster 1:
Cluster con pocos elementos o poca variedad de stage
Cluster 2:
0:	learn: 0.6341980	test: 0.6342828	best: 0.6342828 (0)	total: 14.9ms	remaining: 89.5ms
1:	learn: 0.5803445	test: 0.5804907	best: 0.5804907 (1)	total: 29.2ms	remaining: 72.9ms
2:	learn: 0.5296029	test: 0.5538787	best: 0.5538787 (2)	total: 42.2ms	remaining: 56.2ms
3:	l

In [11]:
#RECONSTRUCCION

for i in range (1, N_CLUSTERS):
    slice_test[0].append(slice_test[i])
    preds[0].append(preds[i])
    slice_stage_test[0].append(slice_stage_test[i])

test = slice_test[0]
preds =  preds[0]
test_label = slice_stage_test[0]

In [12]:
if not (PREDICCION_REAL):
    print(log_loss(test_label, preds))

0.6364916025890319


In [13]:
fallar

NameError: name 'fallar' is not defined

In [ ]:
resultados = pd.DataFrame()
resultados['Opportunity_ID'] = test['Opportunity_ID']
resultados['Target'] = pd.Series(preds)
resultados = resultados.groupby('Opportunity_ID').mean()
resultados = resultados.reset_index()

if not (PREDICCION_REAL):
    resultados.to_csv("../Archivos/prediccion_cluster_validacion.csv", index=False)
else:
    resultados.to_csv("../Archivos/prediccion_cluster_test.csv", index=False)
    
resultados['Target'].value_counts()